In [61]:
import pandas as pd
import pymysql
from datetime import datetime
import calendar
from datetime import timedelta

In [62]:
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="123456",
    database="walmart_dw"
)
cursor = conn.cursor()

In [70]:
customer = pd.read_csv("customer_master_data.csv")
product = pd.read_csv("product_master_data.csv")
transactions = pd.read_csv("transactional_data.csv")

transactions.head()

,Unnamed: 0,orderID,Customer_ID,Product_ID,quantity,date
0,0,9914432,1000001,P00069042,1,2020-12-29
1,1,1676537,1000001,P00248942,3,2017-03-27
2,2,8910457,1000001,P00087842,1,2017-08-20
3,3,5044982,1000001,P00085442,1,2019-03-03
4,4,4176351,1000002,P00285442,1,2018-07-11


In [64]:
def seasons(month):
    if month in [3,4,5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'summer'
    elif month in [12, 1, 2]:
        return 'Winter'
    else: 
        return 'Autumn'
        

In [ ]:
for i, row in customer.iterrows():
    cursor.execute(""" insert into customer(customerid, gender, age_range, occupation, city_category, stay_years, martial_status ) 
                       Values(%s, %s, %s, %s, %s, %s, %s)""",

                       (row.Customer_ID, row.Gender, row.Age, row.Occupation, row.City_Category, row.Stay_In_Current_City_Years, row.Marital_Status)
                  )
conn.commit()
print("Customer Master Data loaded")

In [55]:
suppliers = product[['supplierID', 'supplierName']].drop_duplicates()


for i, row in suppliers.iterrows():
    cursor.execute(""" insert ignore into supplier(supplierid, supplierName) 
                       Values(%s, %s)""",

                       (row.supplierID, row.supplierName)
                  )
conn.commit()
print("Supplier Data loaded")

Supplier Data loaded


In [56]:
stores = product[['storeID', 'storeName']].drop_duplicates()


for i, row in stores.iterrows():
    cursor.execute(""" insert ignore into store(storeid, storeName) 
                       Values(%s, %s)""",

                       (row.storeID, row.storeName)
                  )
conn.commit()
print("Supplier Data loaded")

Supplier Data loaded


In [57]:
for i, row in product.iterrows():
    cursor.execute(""" insert into product(productid, productCategory, price, supplierid, storeid) 
                       Values(%s, %s, %s, %s, %s)""",

                       (row.Product_ID, row.Product_Category, row['price$'], row.supplierID, row.storeID)
                  )
conn.commit()
print("Product Data loaded")

Product Data loaded


In [71]:
from datetime import timedelta
import pandas as pd


transactions['date'] = pd.to_datetime(transactions['date'], format='%Y-%m-%d')

min_date = transactions['date'].min()
max_date = transactions['date'].max()

print("Min date:", min_date)
print("Max date:", max_date)

start_date = min_date - timedelta(days=365)
end_date = max_date + timedelta(days=365)

print("Start date:", start_date)
print("End date:", end_date)


Min date: 2015-01-01 00:00:00
Max date: 2020-12-31 00:00:00
Start date: 2014-01-01 00:00:00
End date: 2021-12-31 00:00:00


In [79]:
current_date = start_date
rows = []

while current_date <= end_date:
    
    weekday = calendar.day_name[current_date.weekday()]  # Monday, Tuesday...
    weekend_flag = 1 if weekday in ['Saturday', 'Sunday'] else 0
    quarter = (current_date.month - 1)//3 + 1
    season = seasons(current_date.month)
    half = 'H1' if current_date.month <= 6 else 'H2'

    rows.append((
        current_date.date(),     
        current_date.day,
        current_date.month,
        quarter,
        current_date.year,
        weekday,
        weekend_flag,
        season,
        half
    ))
    current_date += timedelta(days=1)

insert_query = """
INSERT IGNORE INTO dateDim
(date, day, month, quarter, year, weekday, weekend_flag, season, half_year)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

cursor.executemany(insert_query, rows)
conn.commit()
print("✅ dim_date populated successfully!")

cursor.close()
conn.close()

✅ dim_date populated successfully!
